<a href="https://colab.research.google.com/github/aalvinsu/Alvin-Public/blob/main/h8dsft_P0W3_Alvin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPOR DATA

In [132]:
from google.colab import auth
from google.cloud import bigquery

# Autentikasi akun Google
auth.authenticate_user()

# Membuat klien BigQuery
client = bigquery.Client(project='iconic-elevator-196204')

## EXPLOR DATA ANALIsis

In [133]:
# mencari Tahu Data Ini Memiliki Table Apa Saja
query = '''
SELECT
  table_name
FROM
  `bigquery-public-data.thelook_ecommerce.INFORMATION_SCHEMA.TABLES`
'''

# Eksekusi query dan mendapatkan hasil
query_job = client.query(query)
results = query_job.result()

# Menampilkan hasil
for row in results:
    print(row.table_name)

users
order_items
distribution_centers
inventory_items
products
orders
events


In [134]:
# Menganalisis Table Orders
query = '''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders`
LIMIT 10
'''

df = client.query(query).to_dataframe()

In [135]:
# Menampilkan 5 data pertama
print(df.head())

   order_id  user_id   status gender                       created_at  \
0        13       13  Shipped      F        2021-11-16 13:28:00+00:00   
1        19       18  Shipped      F 2023-06-16 03:01:31.810245+00:00   
2        21       19  Shipped      F        2022-09-15 07:07:00+00:00   
3        23       21  Shipped      F        2021-11-06 14:44:00+00:00   
4        24       22  Shipped      F        2023-05-27 06:51:00+00:00   

  returned_at                       shipped_at delivered_at  num_of_item  
0         NaT        2021-11-17 22:00:00+00:00          NaT            1  
1         NaT 2023-06-16 04:52:31.810245+00:00          NaT            1  
2         NaT        2022-09-16 16:59:00+00:00          NaT            1  
3         NaT        2021-11-08 23:41:00+00:00          NaT            1  
4         NaT        2023-05-28 19:04:00+00:00          NaT            1  


***1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?***

Insight yang bisa diberikan adalah data menunjukkan adanya pertumbuhan pada setiap bulan, dimana memang transaksi dari bulan ke bulan mengalami peningkatan.

In [136]:
# Jumlah transaksi yang berstatus compelet setiap bulan selama Q1-Q3 di tahun 2022

query = '''
SELECT
  EXTRACT(MONTH FROM TIMESTAMP_TRUNC(shipped_at, MONTH)) AS bulan,
  COUNT(*) AS jumlah_transaksi
FROM
  `bigquery-public-data.thelook_ecommerce.orders`
WHERE
  TIMESTAMP_TRUNC(shipped_at, YEAR) = TIMESTAMP("2022-01-01T00:00:00Z")
  AND TIMESTAMP_TRUNC(shipped_at, QUARTER) BETWEEN TIMESTAMP("2022-01-01T00:00:00Z") AND TIMESTAMP("2022-09-30T23:59:59Z")
  AND status = 'Shipped'
GROUP BY
  bulan
ORDER BY
  bulan
'''

# Eksekusi query dan mendapatkan hasil
query_job = client.query(query)
results = query_job.result()

# Menampilkan hasil
for row in results:
    print(f'Bulan: {row.bulan}, Jumlah Transaksi: {row.jumlah_transaksi}')

Bulan: 1, Jumlah Transaksi: 760
Bulan: 2, Jumlah Transaksi: 781
Bulan: 3, Jumlah Transaksi: 871
Bulan: 4, Jumlah Transaksi: 882
Bulan: 5, Jumlah Transaksi: 964
Bulan: 6, Jumlah Transaksi: 961
Bulan: 7, Jumlah Transaksi: 1097
Bulan: 8, Jumlah Transaksi: 1137
Bulan: 9, Jumlah Transaksi: 1251


***Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?)***

Informasi yang bisa saya sampaikan adalah adanya pertumbuhan order yang secara umum terus meningkat menyebabkan penjualan juga meningkat, hanya saja memang terjadi fluktuatif pada bulan tertentu.

In [137]:
# Total penjualan dalam USD setiap bulan selama Q1-Q3
query = '''
SELECT
  EXTRACT(MONTH FROM TIMESTAMP_TRUNC(o.shipped_at, MONTH)) AS bulan,
  SUM(i.cost) AS total_penjualan
FROM
  `bigquery-public-data.thelook_ecommerce.orders` AS o
JOIN
  `bigquery-public-data.thelook_ecommerce.inventory_items` AS i
ON
  o.created_at = i.created_at
WHERE
  TIMESTAMP_TRUNC(o.shipped_at, YEAR) = TIMESTAMP("2022-01-01T00:00:00Z")
  AND TIMESTAMP_TRUNC(o.shipped_at, QUARTER) BETWEEN TIMESTAMP("2022-01-01T00:00:00Z") AND TIMESTAMP("2022-09-30T23:59:59Z")
  AND o.status = 'Shipped'
GROUP BY
  bulan
ORDER BY
  bulan
'''

# Eksekusi query dan mendapatkan hasil
query_job = client.query(query)
results = query_job.result()

# Menampilkan hasil
for row in results:
    print(f'Bulan: {row.bulan}, Total Penjualan: {row.total_penjualan}')

Bulan: 1, Total Penjualan: 3977.7950241328763
Bulan: 2, Total Penjualan: 5060.143262507887
Bulan: 3, Total Penjualan: 4634.83821802552
Bulan: 4, Total Penjualan: 4648.569012535104
Bulan: 5, Total Penjualan: 7292.559485617736
Bulan: 6, Total Penjualan: 5302.233582570643
Bulan: 7, Total Penjualan: 7581.61916142278
Bulan: 8, Total Penjualan: 6344.954737478664
Bulan: 9, Total Penjualan: 8983.815911798938


***Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?***

User secara general mengalami peningkatan dalam bertransaksi, hanya saja pada bulan ke 3 dan 4 mengalami sedikit penurunan yang perlu kita lihat penyebabnya dengan melakukan investigasi baik itu penurunan budget marketing, stock barang yang mungkin menipis, atau adanya kendala eksternal seperti adanya promo competitor yang lebih menggiurkan.

In [138]:
# Data Jumlah User
query = '''
SELECT
  EXTRACT(MONTH FROM TIMESTAMP_TRUNC(o.shipped_at, MONTH)) AS bulan,
  COUNT(DISTINCT o.user_id) AS jumlah_pengguna
FROM
  `bigquery-public-data.thelook_ecommerce.orders` AS o
WHERE
  TIMESTAMP_TRUNC(o.shipped_at, YEAR) = TIMESTAMP("2022-01-01T00:00:00Z")
  AND TIMESTAMP_TRUNC(o.shipped_at, QUARTER) BETWEEN TIMESTAMP("2022-01-01T00:00:00Z") AND TIMESTAMP("2022-09-30T23:59:59Z")
  AND o.status = 'Complete'
GROUP BY
  bulan
ORDER BY
  bulan
'''

# Eksekusi query dan mendapatkan hasil
query_job = client.query(query)
results = query_job.result()

# Menampilkan hasil
for row in results:
    print(f'Bulan: {row.bulan}, Jumlah Pengguna: {row.jumlah_pengguna}')

Bulan: 1, Jumlah Pengguna: 622
Bulan: 2, Jumlah Pengguna: 607
Bulan: 3, Jumlah Pengguna: 746
Bulan: 4, Jumlah Pengguna: 744
Bulan: 5, Jumlah Pengguna: 808
Bulan: 6, Jumlah Pengguna: 857
Bulan: 7, Jumlah Pengguna: 910
Bulan: 8, Jumlah Pengguna: 952
Bulan: 9, Jumlah Pengguna: 1044


***4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!***

***5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!***

Jawaban Nomor 4 dan 5:
Kategori Outerwear & Coats, Pendapatan: 1002.9289868644682 pada bulan ke-6.
Pada bulan ini terjadi peningkatabn pembelian Outerwear dan Coast karena pada bulan Juni tengah memasukki peralihan musim dari summe meuju auntum dimana udara akan terasa lebih dingin sehingga konsumen menyiapkan diri menghadapi musim gugur yang kemudian akan diikuti musim dingin.

In [139]:
query = '''
SELECT
  EXTRACT(MONTH FROM TIMESTAMP_TRUNC(o.created_at, MONTH)) AS bulan,
  i.product_category AS kategori_produk,
  SUM(i.cost) AS total_pendapatan
FROM
  `bigquery-public-data.thelook_ecommerce.orders` AS o
JOIN
  `bigquery-public-data.thelook_ecommerce.inventory_items` AS i
  ON o.created_at = i.created_at
WHERE
  TIMESTAMP_TRUNC(o.created_at, YEAR) = TIMESTAMP("2022-01-01T00:00:00Z")
  AND TIMESTAMP_TRUNC(o.created_at, QUARTER) BETWEEN TIMESTAMP("2022-01-01T00:00:00Z") AND TIMESTAMP("2022-09-30T23:59:59Z")
  AND o.status = 'Complete'
GROUP BY
  bulan, kategori_produk
ORDER BY
  bulan, total_pendapatan DESC
'''

# Eksekusi query dan mendapatkan hasil
query_job = client.query(query)
results = query_job.result()

# Menampilkan hasil
for row in results:
    print(f'Bulan: {row.bulan}, Kategori Produk: {row.kategori_produk}, Total Pendapatan: {row.total_pendapatan}')


Bulan: 1, Kategori Produk: Jeans, Total Pendapatan: 687.843929518921
Bulan: 1, Kategori Produk: Sweaters, Total Pendapatan: 381.66186164451494
Bulan: 1, Kategori Produk: Fashion Hoodies & Sweatshirts, Total Pendapatan: 286.7194337397178
Bulan: 1, Kategori Produk: Intimates, Total Pendapatan: 283.3591402664432
Bulan: 1, Kategori Produk: Swim, Total Pendapatan: 266.10011337783874
Bulan: 1, Kategori Produk: Shorts, Total Pendapatan: 240.11058044420747
Bulan: 1, Kategori Produk: Outerwear & Coats, Total Pendapatan: 233.265300559948
Bulan: 1, Kategori Produk: Tops & Tees, Total Pendapatan: 225.78586132600097
Bulan: 1, Kategori Produk: Dresses, Total Pendapatan: 189.94955008415207
Bulan: 1, Kategori Produk: Sleep & Lounge, Total Pendapatan: 145.29743910748084
Bulan: 1, Kategori Produk: Pants, Total Pendapatan: 139.87573155168536
Bulan: 1, Kategori Produk: Accessories, Total Pendapatan: 127.38729985208138
Bulan: 1, Kategori Produk: Plus, Total Pendapatan: 111.90515957649345
Bulan: 1, Kategori

In [140]:
# Melihat kategori penyumbang omset terbanyak
query = '''
SELECT
  EXTRACT(MONTH FROM TIMESTAMP_TRUNC(o.created_at, MONTH)) AS bulan,
  i.product_category AS kategori_produk,
  SUM(i.cost) AS pendapatan
FROM
  `bigquery-public-data.thelook_ecommerce.orders` AS o
JOIN
  `bigquery-public-data.thelook_ecommerce.inventory_items` AS i
  ON o.created_at = i.created_at
WHERE
  TIMESTAMP_TRUNC(o.created_at, YEAR) = TIMESTAMP("2022-01-01T00:00:00Z")
  AND TIMESTAMP_TRUNC(o.created_at, QUARTER) BETWEEN TIMESTAMP("2022-01-01T00:00:00Z") AND TIMESTAMP("2022-09-30T23:59:59Z")
  AND o.status = 'Complete'
GROUP BY
  bulan, kategori_produk
HAVING
  SUM(i.cost) = (
    SELECT
      MAX(total_pendapatan)
    FROM (
      SELECT
        EXTRACT(MONTH FROM TIMESTAMP_TRUNC(o.created_at, MONTH)) AS bulan,
        i.product_category AS kategori_produk,
        SUM(i.cost) AS total_pendapatan
      FROM
        `bigquery-public-data.thelook_ecommerce.orders` AS o
      JOIN
        `bigquery-public-data.thelook_ecommerce.inventory_items` AS i
        ON o.created_at = i.created_at
      WHERE
        TIMESTAMP_TRUNC(o.created_at, YEAR) = TIMESTAMP("2022-01-01T00:00:00Z")
        AND TIMESTAMP_TRUNC(o.created_at, QUARTER) BETWEEN TIMESTAMP("2022-01-01T00:00:00Z") AND TIMESTAMP("2022-09-30T23:59:59Z")
        AND o.status = 'Complete'
      GROUP BY
        bulan, kategori_produk
    ) AS subquery
    WHERE subquery.bulan = bulan
  )
ORDER BY
  bulan
'''

# Eksekusi query dan mendapatkan hasil
query_job = client.query(query)
results = query_job.result()

# Menampilkan hasil
for row in results:
    print(f'Bulan: {row.bulan}, Kategori Produk: {row.kategori_produk}, Pendapatan: {row.pendapatan}')

Bulan: 6, Kategori Produk: Outerwear & Coats, Pendapatan: 1002.9289868644682


Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan?

Jika kita melihat pada data transaksi pada bulan ke 3 dan 4, ternyata mengalami penurunan walaupun terjadi peningkatan penjualan. Hal ini menunjukkan bahwa jumlah user tidak signifikan berpengaruh pada peningkatan penjualan.

***Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)***

Kesimpulannya adalah bahwa jumlah transaksi tidak berhubungan dengan jumlah user dimana memang pada user yang mungkin datang ke website ecommerce ini, tetapi tidak melakukan pembelian.

Guna mengubah user yang tidak melakukan pembelian ini menjadi pembeli harus ditingkatkan conversion ratenya, seperti menawarkan promo lebih menarik stok produk lebih bervariatif, dan juga memberikan diskon khusus untuk mereka agar mau bertransaksi

***Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!***

Dapat, kita bisa melihat jumlah pertumbuhan penjualan, pendapatan, hingga revenue dari perusahaan

***Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250,000, apakah masuk akal?***

Menurut saya tidak masuk akal, karena dari historical data yang ada belum pernah company mencapai target tersebut bahkan mendekati angka tersebut.

***CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!***

Menurut saya layak karena trend pertumbuhan user cenderung positif, dan angka penjualan memiliki fluktuatif, namun trend ke arah positif. Hal ini membuat CEO bisa menggunakan data investasi untuk melakukan penjualan produk lebih bervariatif, membuat promo lebih menarik dan meningkatkan kualitas dari website agar user mau menjadi buyer.